# Canadian Wage Analysis Project

*For the resulting Tableau Visualization: https://public.tableau.com/app/profile/murad5017/viz/CanadianWageProject/Story1* 

**Background**: Every end of year, the federal government of Canada shares the low, median, and high estimations of the wage per hour for every occupation in every province and territory. For example, the data for Ontario can be found here: https://www.jobbank.gc.ca/wagereport/location/on

**Problem**: The wage reports are separate and do not compare provincial trends even though the data was collected and built on the provincial and territorial level. There is, accordingly, a lack of contextualization of discrepancies among provinces. Such that we are not able to see how differences among provinces can be mirrored in difference in wage per hour. Additionally, the released data is for each and every specific occupation. It doesn't display the different job hierarchy and classifications (NOC) as shown here: https://noc.esdc.gc.ca/Structure/Hierarchy?objectid=%2Fd0IGA6qD8JPRfoj5UCjpg%3D%3D

**Approach**: We are going to collect all the reports for the provinces in the year 2023, merge them, contextualize provincial differences (minimum wage, % of national GDP, population) as well as explore how the wages differ by different occupation hierarchies. We use Python for data cleaning and wrangling. Tableau is used for the visualization of the final data.

Once more, for the resulting Tableau Visualization, kindly click this link (best viewed on full screen): https://public.tableau.com/app/profile/murad5017/viz/CanadianWageProject/Story1

## 1) Reading data and variable creation

In [1]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = r'C:\Users\Murad\Music\dtdb\SQL projects\Canada_Wage_Data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path, encoding='latin-1')

# Display the DataFrame
print(df)

                                       Occupation (NOC)     Low  Median  \
0                                   Legislators (00010)  35,320  80,000   
1      Senior government managers and officials (00011)      20   65.76   
2     Senior managers - financial, communications an...      44   84.13   
3     Senior managers - trade, broadcasting and othe...      23   46.16   
4     Senior managers - construction, transportation...   21.54    45.7   
...                                                 ...     ...     ...   
4741                             Mine labourers (85110)      23   28.99   
4742  Landscaping and grounds maintenance labourers ...      21   27.07   
4743                         Utilities managers (90011)   30.34   49.61   
4744  Power engineers and power systems operators (9...   28.25   46.78   
4745  Water and waste treatment plant operators (92101)   21.61   32.78   

         High Province  
0     186,000       ON  
1       96.63       ON  
2      137.36       ON  

From the above snapshot of the data, we see that the "Occupation (NOC)" column is actually two merged into one. Each row of that column has the occupation name as well as its specific occupation code. For better analysis, we should separate them into two columns.

In [2]:
# Split the column based on the '(' character
df[['Occupation', 'NOC']] = df['Occupation (NOC)'].str.rsplit('(', 1, expand=True)

# Drop the original column if needed
df = df.drop('Occupation (NOC)', axis=1)

# Display the result
print(df)

         Low  Median     High Province  \
0     35,320  80,000  186,000       ON   
1         20   65.76    96.63       ON   
2         44   84.13   137.36       ON   
3         23   46.16   105.64       ON   
4      21.54    45.7   102.56       ON   
...      ...     ...      ...      ...   
4741      23   28.99     34.6       YT   
4742      21   27.07    30.61       YT   
4743   30.34   49.61    84.09       YT   
4744   28.25   46.78    67.93       YT   
4745   21.61   32.78    52.37       YT   

                                             Occupation     NOC  
0                                          Legislators   00010)  
1             Senior government managers and officials   00011)  
2     Senior managers - financial, communications an...  00012)  
3     Senior managers - trade, broadcasting and othe...  00014)  
4     Senior managers - construction, transportation...  00015)  
...                                                 ...     ...  
4741                             

In [3]:


# Remove only the parentheses from the 'NOC' values
df['NOC'] = df['NOC'].str.replace(r'[()]', '', regex=True)

# Display the result
print(df)

         Low  Median     High Province  \
0     35,320  80,000  186,000       ON   
1         20   65.76    96.63       ON   
2         44   84.13   137.36       ON   
3         23   46.16   105.64       ON   
4      21.54    45.7   102.56       ON   
...      ...     ...      ...      ...   
4741      23   28.99     34.6       YT   
4742      21   27.07    30.61       YT   
4743   30.34   49.61    84.09       YT   
4744   28.25   46.78    67.93       YT   
4745   21.61   32.78    52.37       YT   

                                             Occupation    NOC  
0                                          Legislators   00010  
1             Senior government managers and officials   00011  
2     Senior managers - financial, communications an...  00012  
3     Senior managers - trade, broadcasting and othe...  00014  
4     Senior managers - construction, transportation...  00015  
...                                                 ...    ...  
4741                                    

We finally separated the NOC portion from the Occupation column. Now, we need to broaden the occupation categories so we can explore any potential differences among occupation groups later on. According to the National Occupation Classification (NOC) set by the federal Canadian government, the first digit of the NOC designates a general occupational group. Those are one of ten groups that cover the different general "fields" or occupation areas of expertise. They include 

(0) legislative and senior management occupations

(1) business, finance and administration occupations

(2) natural and applied sciences and related occupations

(3) health occupations

(4) occupations in education, law and social, community and government services

(5) occupations in art, culture, recreation and sport

(6) sales and service occupations

(7) trades, transport and equipment operators and related occupations

(8) natural resources, agricultrue and related production occupations

(9) occupations in manufacturing and utilities

Each of the digits represents the first digit of the NOC code. Accordingly, we will create a new variable for just the first NOC code and we will also add these labels/descriptions so the codes are understandable when visualized.

In [4]:
# Define a function to extract the first digit from a string
def extract_first_character(value):
    return value[0]

# Create a new column based on the first digit of 'OldColumn'
df['Occupation_Code'] = df['NOC'].apply(extract_first_character)

# Display the result
print(df)

         Low  Median     High Province  \
0     35,320  80,000  186,000       ON   
1         20   65.76    96.63       ON   
2         44   84.13   137.36       ON   
3         23   46.16   105.64       ON   
4      21.54    45.7   102.56       ON   
...      ...     ...      ...      ...   
4741      23   28.99     34.6       YT   
4742      21   27.07    30.61       YT   
4743   30.34   49.61    84.09       YT   
4744   28.25   46.78    67.93       YT   
4745   21.61   32.78    52.37       YT   

                                             Occupation    NOC Occupation_Code  
0                                          Legislators   00010               0  
1             Senior government managers and officials   00011               0  
2     Senior managers - financial, communications an...  00012               0  
3     Senior managers - trade, broadcasting and othe...  00014               0  
4     Senior managers - construction, transportation...  00015               0  
...      

In [5]:
df['Occupation_Code'] = df['Occupation_Code'].astype(int)
df

,Low,Median,High,Province,Occupation,NOC,Occupation_Code
0,"35,320","80,000","186,000",ON,Legislators,00010,0
1,20,65.76,96.63,ON,Senior government managers and officials,00011,0
2,44,84.13,137.36,ON,"Senior managers - financial, communications an...",00012,0
3,23,46.16,105.64,ON,"Senior managers - trade, broadcasting and othe...",00014,0
4,21.54,45.7,102.56,ON,"Senior managers - construction, transportation...",00015,0
...,...,...,...,...,...,...,...
4741,23,28.99,34.6,YT,Mine labourers,85110,8
4742,21,27.07,30.61,YT,Landscaping and grounds maintenance labourers,85121,8
4743,30.34,49.61,84.09,YT,Utilities managers,90011,9
4744,28.25,46.78,67.93,YT,Power engineers and power systems operators,92100,9


In [6]:
# Define a function to extract the first two digits from a string
def extract_firsttwo_character(value):
    return value[:2]

# Create a new column based on the first digit of 'OldColumn'
df['Occupation_Code_2'] = df['NOC'].apply(extract_firsttwo_character)

# Display the result
print(df)

         Low  Median     High Province  \
0     35,320  80,000  186,000       ON   
1         20   65.76    96.63       ON   
2         44   84.13   137.36       ON   
3         23   46.16   105.64       ON   
4      21.54    45.7   102.56       ON   
...      ...     ...      ...      ...   
4741      23   28.99     34.6       YT   
4742      21   27.07    30.61       YT   
4743   30.34   49.61    84.09       YT   
4744   28.25   46.78    67.93       YT   
4745   21.61   32.78    52.37       YT   

                                             Occupation    NOC  \
0                                          Legislators   00010   
1             Senior government managers and officials   00011   
2     Senior managers - financial, communications an...  00012   
3     Senior managers - trade, broadcasting and othe...  00014   
4     Senior managers - construction, transportation...  00015   
...                                                 ...    ...   
4741                             

In [7]:
#mapping to broader NOC code descriptions

occupation_mapping = {
    0: 'Legislative and senior management occupations',
    1: 'Business, finance and administration occupations',
    2: 'Natural and applied sciences and related occupations',
    3: 'Health occupations',
    4: 'Occupations in education, law and social, community and government services',
    5: 'Occupations in art, culture, recreation and sport',
    6: 'Sales and service occupations',
    7: 'Trades, transport and equipment operators and related occupations',
    8: 'Natural resources, agriculture and related production occupations',
    9: 'Occupations in manufacturing and utilities'
}

# Replace values in 'OccupationCode' based on the dictionary mapping
df['OccupationDescription'] = df['Occupation_Code'].replace(occupation_mapping)
df

,Low,Median,High,Province,Occupation,NOC,Occupation_Code,Occupation_Code_2,OccupationDescription
0,"35,320","80,000","186,000",ON,Legislators,00010,0,00,Legislative and senior management occupations
1,20,65.76,96.63,ON,Senior government managers and officials,00011,0,00,Legislative and senior management occupations
2,44,84.13,137.36,ON,"Senior managers - financial, communications an...",00012,0,00,Legislative and senior management occupations
3,23,46.16,105.64,ON,"Senior managers - trade, broadcasting and othe...",00014,0,00,Legislative and senior management occupations
4,21.54,45.7,102.56,ON,"Senior managers - construction, transportation...",00015,0,00,Legislative and senior management occupations
...,...,...,...,...,...,...,...,...,...
4741,23,28.99,34.6,YT,Mine labourers,85110,8,85,"Natural resources, agriculture and related pro..."
4742,21,27.07,30.61,YT,Landscaping and grounds maintenance labourers,85121,8,85,"Natural resources, agriculture and related pro..."
4743,30.34,49.61,84.09,YT,Utilities managers,90011,9,90,Occupations in manufacturing and utilities
4744,28.25,46.78,67.93,YT,Power engineers and power systems operators,92100,9,92,Occupations in manufacturing and utilities


Now that we have broadened the occupation breakdown. We look at other different occupation groupings. This time we take the first two digits of the NOC code. The mapping used below is based on the breakdown provided by the government of Canada.

Note: This level of grouping is not used for visualization as its results were not deemed fruitful

In [8]:
#secondary NOC code mapping

occupation_mapping_2 = {
    "00": "Legislative and senior managers",
    "10": "Specialized middle management occupations in administrative services, financial and business services and communication (except broadcasting)",
    "11": "Professional occupations in finance and business",
    "12": "Administrative and financial supervisors and specialized administrative occupations",
    "13": "Administrative occupations and transportation logistics occupations",
    "14": "Administrative and financial support and supply chain logistics occupations",
    "20": "Specialized middle management occupations in engineering, architecture, science and information systems",
    "21": "Professional occupations in natural and applied sciences",
    "22": "Technical occupations related to natural and applied sciences",
    "30": "Specialized middle management occupations in health care",
    "31": "Professional occupations in health",
    "32": "Technical occupations in health",
    "33": "Assisting occupations in support of health services",
    "40": "Managers in public administration, in education and social and community services and in public protection services",
    "41": "Professional occupations in law, education, social, community and government services",
    "42": "Front-line public protection services and paraprofessional occupations in legal, social, community, education services",
    "43": "Assisting occupations in education and in legal and public protection",
    "44": "Care providers and public protection support occupations",
    "45": "Student monitors, crossing guards and related occupations",
    "50": "Specialized middle management occupations in art, culture, recreation and sport",
    "51": "Professional occupations in art and culture",
    "52": "Technical occupations in art, culture and sport",
    "53": "Occupations in art, culture and sport",
    "54": "Support occupations in sport",
    "55": "Support occupations in art and culture",
    "60": "Middle management occupations in retail and wholesale trade and customer services",
    "62": "Retail sales and service supervisors and specialized occupations in sales and services",
    "63": "Occupations in sales and services",
    "64": "Sales and service representatives and other customer and personal services occupations",
    "65": "Sales and service support occupations",
    "70": "Middle management occupations in trades and transportation",
    "72": "Technical trades and transportation officers and controllers",
    "73": "General trades",
    "74": "Mail and message distribution, other transport equipment operators and related maintenance workers",
    "75": "Helpers and labourers and other transport drivers, operators and labourers",
    "80": "Middle management occupations in production and agriculture",
    "82": "Supervisors in natural resources, agriculture and related production",
    "83": "Occupations in natural resources and related production",
    "84": "Workers in natural resources, agriculture and related production",
    "85": "Harvesting, landscaping and natural resources labourers",
    "90": "Middle management occupations in manufacturing and utilities",
    "92": "Processing, manufacturing and utilities supervisors and utilities operators and controllers",
    "93": "Central control and process operators and aircraft assembly assemblers and inspectors",
    "94": "Machine operators, assemblers and inspectors in processing, manufacturing and printing",
    "95": "Labourers in processing, manufacturing and utilities"
}

df['OccupationDescription_2'] = df['Occupation_Code_2'].replace(occupation_mapping_2)
df

,Low,Median,High,Province,Occupation,NOC,Occupation_Code,Occupation_Code_2,OccupationDescription,OccupationDescription_2
0,"35,320","80,000","186,000",ON,Legislators,00010,0,00,Legislative and senior management occupations,Legislative and senior managers
1,20,65.76,96.63,ON,Senior government managers and officials,00011,0,00,Legislative and senior management occupations,Legislative and senior managers
2,44,84.13,137.36,ON,"Senior managers - financial, communications an...",00012,0,00,Legislative and senior management occupations,Legislative and senior managers
3,23,46.16,105.64,ON,"Senior managers - trade, broadcasting and othe...",00014,0,00,Legislative and senior management occupations,Legislative and senior managers
4,21.54,45.7,102.56,ON,"Senior managers - construction, transportation...",00015,0,00,Legislative and senior management occupations,Legislative and senior managers
...,...,...,...,...,...,...,...,...,...,...
4741,23,28.99,34.6,YT,Mine labourers,85110,8,85,"Natural resources, agriculture and related pro...","Harvesting, landscaping and natural resources ..."
4742,21,27.07,30.61,YT,Landscaping and grounds maintenance labourers,85121,8,85,"Natural resources, agriculture and related pro...","Harvesting, landscaping and natural resources ..."
4743,30.34,49.61,84.09,YT,Utilities managers,90011,9,90,Occupations in manufacturing and utilities,Middle management occupations in manufacturing...
4744,28.25,46.78,67.93,YT,Power engineers and power systems operators,92100,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv..."


Our last but certainly not least occupational breakdown would be the classification level variable. Instead of extrapolating an additional hierarchy from the first digit of the NOC, we do so from the last digit. The last digit, for all occupations and occupation groups, doesn't have to do with fields or specializations but rather a classification level based on managerial and educational credentials, regardless of the field. It's more based on "position" than specific "occupation". The position classification is as follows:

(0) Managers

(1) Professionals

(2) Technical Occupations and/or Supervisors

(3) General Occupations/Trades

(4) Workers/Machine Operators/Support

(5) Labourers/Drivers/Support

Each of the digits represents the last digit of the NOC code. Accordingly, we will create a new variable for just the last NOC code and we will also add these labels/descriptions so the codes are understandable when visualized.


In [9]:
df['Position_Code'] = df['Occupation_Code_2'].str[1:].astype(int)
df

,Low,Median,High,Province,Occupation,NOC,Occupation_Code,Occupation_Code_2,OccupationDescription,OccupationDescription_2,Position_Code
0,"35,320","80,000","186,000",ON,Legislators,00010,0,00,Legislative and senior management occupations,Legislative and senior managers,0
1,20,65.76,96.63,ON,Senior government managers and officials,00011,0,00,Legislative and senior management occupations,Legislative and senior managers,0
2,44,84.13,137.36,ON,"Senior managers - financial, communications an...",00012,0,00,Legislative and senior management occupations,Legislative and senior managers,0
3,23,46.16,105.64,ON,"Senior managers - trade, broadcasting and othe...",00014,0,00,Legislative and senior management occupations,Legislative and senior managers,0
4,21.54,45.7,102.56,ON,"Senior managers - construction, transportation...",00015,0,00,Legislative and senior management occupations,Legislative and senior managers,0
...,...,...,...,...,...,...,...,...,...,...,...
4741,23,28.99,34.6,YT,Mine labourers,85110,8,85,"Natural resources, agriculture and related pro...","Harvesting, landscaping and natural resources ...",5
4742,21,27.07,30.61,YT,Landscaping and grounds maintenance labourers,85121,8,85,"Natural resources, agriculture and related pro...","Harvesting, landscaping and natural resources ...",5
4743,30.34,49.61,84.09,YT,Utilities managers,90011,9,90,Occupations in manufacturing and utilities,Middle management occupations in manufacturing...,0
4744,28.25,46.78,67.93,YT,Power engineers and power systems operators,92100,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv...",2


In [10]:
# Define a function to map values to categories

def map_to_category(value):
    if value == 0:
        return "Managers"
    elif value == 1:
        return "Professionals"
    elif value == 2:
        return "Technical Occupations and/or Supervisors"
    elif value == 3:
        return "General Occupations/Trades"
    elif value == 4:
        return "Workers/Machine Operators/Support"
    elif value == 5:
        return "Labourers/Drivers/Support"
    else:
        return "Unknown"
    
df['Position'] = df['Position_Code'].apply(map_to_category)
df

,Low,Median,High,Province,Occupation,NOC,Occupation_Code,Occupation_Code_2,OccupationDescription,OccupationDescription_2,Position_Code,Position
0,"35,320","80,000","186,000",ON,Legislators,00010,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
1,20,65.76,96.63,ON,Senior government managers and officials,00011,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
2,44,84.13,137.36,ON,"Senior managers - financial, communications an...",00012,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
3,23,46.16,105.64,ON,"Senior managers - trade, broadcasting and othe...",00014,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
4,21.54,45.7,102.56,ON,"Senior managers - construction, transportation...",00015,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
...,...,...,...,...,...,...,...,...,...,...,...,...
4741,23,28.99,34.6,YT,Mine labourers,85110,8,85,"Natural resources, agriculture and related pro...","Harvesting, landscaping and natural resources ...",5,Labourers/Drivers/Support
4742,21,27.07,30.61,YT,Landscaping and grounds maintenance labourers,85121,8,85,"Natural resources, agriculture and related pro...","Harvesting, landscaping and natural resources ...",5,Labourers/Drivers/Support
4743,30.34,49.61,84.09,YT,Utilities managers,90011,9,90,Occupations in manufacturing and utilities,Middle management occupations in manufacturing...,0,Managers
4744,28.25,46.78,67.93,YT,Power engineers and power systems operators,92100,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv...",2,Technical Occupations and/or Supervisors


In [11]:
#removing string characters
df['Low'] = df['Low'].str.replace(',', '', regex=True)
df['Median'] = df['Median'].str.replace(',', '', regex=True)
df['High'] = df['High'].str.replace(',', '', regex=True)

#transforming to float (numeric)
df['Low'] = df['Low'].astype(float)
df['Median'] = df['Median'].astype(float)
df['High'] = df['High'].astype(float)

df

,Low,Median,High,Province,Occupation,NOC,Occupation_Code,Occupation_Code_2,OccupationDescription,OccupationDescription_2,Position_Code,Position
0,35320.00,80000.00,186000.00,ON,Legislators,00010,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
1,20.00,65.76,96.63,ON,Senior government managers and officials,00011,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
2,44.00,84.13,137.36,ON,"Senior managers - financial, communications an...",00012,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
3,23.00,46.16,105.64,ON,"Senior managers - trade, broadcasting and othe...",00014,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
4,21.54,45.70,102.56,ON,"Senior managers - construction, transportation...",00015,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
...,...,...,...,...,...,...,...,...,...,...,...,...
4741,23.00,28.99,34.60,YT,Mine labourers,85110,8,85,"Natural resources, agriculture and related pro...","Harvesting, landscaping and natural resources ...",5,Labourers/Drivers/Support
4742,21.00,27.07,30.61,YT,Landscaping and grounds maintenance labourers,85121,8,85,"Natural resources, agriculture and related pro...","Harvesting, landscaping and natural resources ...",5,Labourers/Drivers/Support
4743,30.34,49.61,84.09,YT,Utilities managers,90011,9,90,Occupations in manufacturing and utilities,Middle management occupations in manufacturing...,0,Managers
4744,28.25,46.78,67.93,YT,Power engineers and power systems operators,92100,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv...",2,Technical Occupations and/or Supervisors


In [12]:
#check missing wage values (if they exist)
x = df[df['Low'].isna()]
x

,Low,Median,High,Province,Occupation,NOC,Occupation_Code,Occupation_Code_2,OccupationDescription,OccupationDescription_2,Position_Code,Position
166,NaN,35320.00,NaN,ON,Traditional Chinese medicine practitioners and...,32200,3,32,Health occupations,Technical occupations in health,2,Technical Occupations and/or Supervisors
315,NaN,35320.00,NaN,ON,Other support occupations in personal services,65229,6,65,Sales and service occupations,Sales and service support occupations,5,Labourers/Drivers/Support
376,NaN,40.51,NaN,ON,"Engineer officers, water transport",72603,7,72,"Trades, transport and equipment operators and ...",Technical trades and transportation officers a...,2,Technical Occupations and/or Supervisors
420,NaN,41.29,NaN,ON,"Contractors and supervisors, oil and gas drill...",82021,8,82,"Natural resources, agriculture and related pro...","Supervisors in natural resources, agriculture ...",2,Technical Occupations and/or Supervisors
455,NaN,34.81,NaN,ON,"Pulping, papermaking and coating control opera...",93102,9,93,Occupations in manufacturing and utilities,Central control and process operators and airc...,3,General Occupations/Trades
...,...,...,...,...,...,...,...,...,...,...,...,...
4717,NaN,29.41,NaN,YT,Plumbers,72300,7,72,"Trades, transport and equipment operators and ...",Technical trades and transportation officers a...,2,Technical Occupations and/or Supervisors
4720,NaN,34.83,NaN,YT,Heavy-duty equipment mechanics,72401,7,72,"Trades, transport and equipment operators and ...",Technical trades and transportation officers a...,2,Technical Occupations and/or Supervisors
4723,NaN,36.46,NaN,YT,"Air pilots, flight engineers and flying instru...",72600,7,72,"Trades, transport and equipment operators and ...",Technical trades and transportation officers a...,2,Technical Occupations and/or Supervisors
4725,NaN,30.48,50.65,YT,General building maintenance workers and build...,73201,7,73,"Trades, transport and equipment operators and ...",General trades,3,General Occupations/Trades


## 2) Data cleaning and exploration

Dealing with outliers and NA values

In [13]:
# filtering to see which professions are shown in annual salary  and not hourly wage
df_2 = df[df['Median'] > 1000] # checking for outlier values
selected_df2 = df_2[['Median', 'Occupation']]
selected_df2['Occupation'].value_counts()


Judges                                                               13
Legislators                                                          12
Real estate agents and salespersons                                  11
Home building and renovation managers                                11
Massage therapists                                                   10
Dentists                                                              9
Specialists in clinical and laboratory medicine                       9
General practitioners and family physicians                           9
Musicians and singers                                                 9
Specialists in surgery                                                9
Chiropractors                                                         8
Optometrists                                                          7
Denturists                                                            5
Other practitioners of natural healing                          

In [14]:
selected_df2['Occupation'].nunique()

18

In [15]:
# replace outliers by average number of hours worked weekly as per Statistics Canada for 2022 (35.8 x 52 weeks = 1861.6 hours)
df.loc[df['Low'] > 1000, 'Low'] /= 1861.6
df.loc[df['Median'] > 1000, 'Median'] /= 1861.6
df.loc[df['High'] > 1000, 'High'] /= 1861.6

In [16]:
df = df.round(2)
df

,Low,Median,High,Province,Occupation,NOC,Occupation_Code,Occupation_Code_2,OccupationDescription,OccupationDescription_2,Position_Code,Position
0,18.97,42.97,99.91,ON,Legislators,00010,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
1,20.00,65.76,96.63,ON,Senior government managers and officials,00011,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
2,44.00,84.13,137.36,ON,"Senior managers - financial, communications an...",00012,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
3,23.00,46.16,105.64,ON,"Senior managers - trade, broadcasting and othe...",00014,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
4,21.54,45.70,102.56,ON,"Senior managers - construction, transportation...",00015,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers
...,...,...,...,...,...,...,...,...,...,...,...,...
4741,23.00,28.99,34.60,YT,Mine labourers,85110,8,85,"Natural resources, agriculture and related pro...","Harvesting, landscaping and natural resources ...",5,Labourers/Drivers/Support
4742,21.00,27.07,30.61,YT,Landscaping and grounds maintenance labourers,85121,8,85,"Natural resources, agriculture and related pro...","Harvesting, landscaping and natural resources ...",5,Labourers/Drivers/Support
4743,30.34,49.61,84.09,YT,Utilities managers,90011,9,90,Occupations in manufacturing and utilities,Middle management occupations in manufacturing...,0,Managers
4744,28.25,46.78,67.93,YT,Power engineers and power systems operators,92100,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv...",2,Technical Occupations and/or Supervisors


In [17]:
#exporting data
df.to_csv('Canada_Wage_Data_V2.csv', index=False)

Going to merge current data with province level data so we can be able to contextualize province and territory difference

In [2]:
# adding index for merging with province level data
df['ID'] = range(1, len(df) + 1)




NameError: name 'df' is not defined

Changing the data format from wide to long so it'll be easier to process and visualize on Tableau

In [18]:
wide_columns = ['Low', 'Median', 'High']
long_partial_df = pd.melt(df, id_vars='ID', value_vars=wide_columns,
                   var_name='Wave_Level', value_name='Value')

In [19]:
partial_df = df[[col for col in df.columns if col not in wide_columns]]
long_df = pd.merge(partial_df, long_partial_df, on='ID', how='inner')  # 'inner' merge keeps only common IDs

In [20]:
long_df

,Province,Occupation,NOC,Occupation_Code,Occupation_Code_2,OccupationDescription,OccupationDescription_2,Position_Code,Position,ID,Wave_Level,Value
0,ON,Legislators,00010,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers,1,Low,18.97
1,ON,Legislators,00010,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers,1,Median,42.97
2,ON,Legislators,00010,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers,1,High,99.91
3,ON,Senior government managers and officials,00011,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers,2,Low,20.00
4,ON,Senior government managers and officials,00011,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers,2,Median,65.76
...,...,...,...,...,...,...,...,...,...,...,...,...
14233,YT,Power engineers and power systems operators,92100,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv...",2,Technical Occupations and/or Supervisors,4745,Median,46.78
14234,YT,Power engineers and power systems operators,92100,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv...",2,Technical Occupations and/or Supervisors,4745,High,67.93
14235,YT,Water and waste treatment plant operators,92101,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv...",2,Technical Occupations and/or Supervisors,4746,Low,21.61
14236,YT,Water and waste treatment plant operators,92101,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv...",2,Technical Occupations and/or Supervisors,4746,Median,32.78


In [28]:
#exporting long partial data to be joined on Tableau
long_partial_df.to_csv('Canada_Wage_Data_lp.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'Canada_Wage_Data_lp.csv'

In [21]:
#adding province-level data

csv_file_path = r'C:\Users\Murad\Music\dtdb\SQL projects\province_level_data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path, encoding='latin-1')

# Display the DataFrame
print(df)

   Province  mean_temperature  Average_house_price  bachelors_higher  \
0        ON              -4.1             0.881946              29.9   
1        BC               4.6             0.960067              28.6   
2        NS              -3.3             0.376600              24.3   
3        NU             -25.2             0.706950              10.6   
4        AB             -11.0             0.446263              25.6   
5        MB             -14.3             0.338022              22.9   
6        NB              -7.7             0.270900              19.0   
7        NL              -3.6             0.272600              16.6   
8        NT             -23.4             0.422787              22.2   
9        PE              -6.1             0.334900              22.0   
10       QC               4.2             0.471460              23.5   
11       SK             -12.9             0.321400              20.5   
12       YT             -13.4             0.519190              

In [22]:
#merging long df with new province-level df

new_df = pd.merge(long_df, df, on='Province', how='inner') 

In [23]:
new_df

,Province,Occupation,NOC,Occupation_Code,Occupation_Code_2,OccupationDescription,OccupationDescription_2,Position_Code,Position,ID,Wave_Level,Value,mean_temperature,Average_house_price,bachelors_higher,Minimum_wage,CPI
0,ON,Legislators,00010,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers,1,Low,18.97,-4.1,0.881946,29.9,16.55,3.4
1,ON,Legislators,00010,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers,1,Median,42.97,-4.1,0.881946,29.9,16.55,3.4
2,ON,Legislators,00010,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers,1,High,99.91,-4.1,0.881946,29.9,16.55,3.4
3,ON,Senior government managers and officials,00011,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers,2,Low,20.00,-4.1,0.881946,29.9,16.55,3.4
4,ON,Senior government managers and officials,00011,0,00,Legislative and senior management occupations,Legislative and senior managers,0,Managers,2,Median,65.76,-4.1,0.881946,29.9,16.55,3.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14233,YT,Power engineers and power systems operators,92100,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv...",2,Technical Occupations and/or Supervisors,4745,Median,46.78,-13.4,0.519190,27.3,16.77,3.0
14234,YT,Power engineers and power systems operators,92100,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv...",2,Technical Occupations and/or Supervisors,4745,High,67.93,-13.4,0.519190,27.3,16.77,3.0
14235,YT,Water and waste treatment plant operators,92101,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv...",2,Technical Occupations and/or Supervisors,4746,Low,21.61,-13.4,0.519190,27.3,16.77,3.0
14236,YT,Water and waste treatment plant operators,92101,9,92,Occupations in manufacturing and utilities,"Processing, manufacturing and utilities superv...",2,Technical Occupations and/or Supervisors,4746,Median,32.78,-13.4,0.519190,27.3,16.77,3.0


In [58]:
#exporting new merged data to be used on Tableau
new_df.to_csv('Canada_Wage_Data_V4.csv', index=False)

Since we're done with data cleaning, exploration, and consolidating, we move load the data on Tableau to visualize the data. Here's the finalized product: https://public.tableau.com/app/profile/murad5017/viz/CanadianWageProject/Story1